In [1]:
import pandas as pd
from geopy.distance import distance
pd.options.display.max_colwidth = 150

In [2]:
df_metro = pd.read_json("../data/input/metro.jsonlines", lines=True)
df_metro

,linha,estacao,lat,lng
0,Linha Paracambi da SuperVia,Estação Paracambi,-22.611839,-43.711019
1,Linha Deodoro da SuperVia,Estação Mangueira (SuperVia),-22.908539,-43.236697
2,Linha Japeri da SuperVia,Estação Engenheiro Pedreira,-22.679417,-43.605161
3,Linha Deodoro da SuperVia,Estação São Francisco Xavier (SuperVia),-22.902703,-43.244172
4,Linha Japeri da SuperVia,Estação São Cristóvão (Rio de Janeiro),-22.909625,-43.223039
...,...,...,...,...
150,Linha Deodoro da SuperVia,Estação Silva Freire,-22.904067,-43.275386
151,Linha Deodoro da SuperVia,Estação Riachuelo,-22.901503,-43.256236
152,Linha Deodoro da SuperVia,Estação Méier,-22.901792,-43.278058
153,Linha Deodoro da SuperVia,Estação Cidade Nova,-22.908889,-43.206389


In [7]:
df = pd.read_json("../data/input/imoveis.jsonlines", lines=True)
df.sample(5)

,id,origem,tipo,metragem,quartos,banheiros,vagas,preco,condominio,lat,lng,endereco,title,desc,url,imagens,url_google
39,2501091787,vivareal,Aluguel,45,1.0,1.0,NaN,1300,650.0,-22.9234,-43.2333,"Rua Santo Afonso - Tijuca, Rio de Janeiro - RJ","Apartamento com 1 Quarto para Alugar, 45 m² por R$ 1.300/Mês","Excelente quarto e sala situado no coração da Tijuca, na mesma quadra do metrô ! Shopping Tijuca a uma quadra de distância! Mercado Mundial e Loja...",https://www.vivareal.com.br/imovel/apartamento-1-quartos-tijuca-zona-norte-rio-de-janeiro-45m2-aluguel-RS1300-id-2501091787/,"[https://resizedimgs.vivareal.com/fit-in/870x653/vr.images.sp/fee841d059cd684f7b927cb8ab293a71.jpg, https://resizedimgs.vivareal.com/crop/142x80/v...","https://maps.google.com/maps?ll=-22.923371,-43.233336&z=16&t=m&hl=en-US&gl=US&mapclient=embed&q=R.%20Santo%20Afonso%20-%20Tijuca%20Rio%20de%20Jane..."
487,2499089019,vivareal,Compra,115,3.0,3.0,1.0,1250000,1400.0,-22.9179,-43.2173,"Rua Campos Sales, 80 - Tijuca, Rio de Janeiro - RJ","Apartamento com 3 Quartos à Venda, 115 m² por R$ 1.250.000","Cód: JCAP30307. Tijuca, praça Afonso Pena, Rua Campos Sales, Maravilhoso apartamento de aproximadamente 115 M², são três quartos, sendo um suíte, ...",https://www.vivareal.com.br/imovel/apartamento-3-quartos-tijuca-zona-norte-rio-de-janeiro-com-garagem-115m2-venda-RS1250000-id-2499089019/,"[https://resizedimgs.vivareal.com/fit-in/870x653/vr.images.sp/630a5f81c91db83906aeb7fc53829051.jpg, https://resizedimgs.vivareal.com/fit-in/870x65...","https://maps.google.com/maps?ll=-22.917897,-43.217311&z=16&t=m&hl=en-US&gl=US&mapclient=embed&q=22%C2%B055%2704.4%22S%2043%C2%B013%2702.3%22W%20-2..."
51,2504630675,zap,Aluguel,58,3.0,1.0,NaN,1500,0.0,-22.9163,-43.2253,"Rua Morais e Silva, 153 - Tijuca, Rio de Janeiro - RJ","Adibrás Excelente localização. Próximo Colégio Militar, c/ vasto comércio e condução. Sala, 3 quarto","Adibrás Excelente localização. Próximo Colégio Militar, c/ vasto comércio e condução. Sala, 3 quartos, cozinha, banheiro, área. Rua Moraes e Silva...",https://www.zapimoveis.com.br/imovel/aluguel-casa-3-quartos-com-area-de-servico-tijuca-zona-norte-rio-de-janeiro-rj-58m2-id-2504630675/,[],"https://maps.google.com/maps?ll=-22.91625,-43.225272&z=17&t=m&hl=en-US&gl=US&mapclient=embed&q=22%C2%B054%2758.5%22S%2043%C2%B013%2731.0%22W%20-22..."
842,2502350716,vivareal,Aluguel,40,1.0,1.0,NaN,1300,592.0,-22.929,-43.2434,"Rua Uruguai - Tijuca, Rio de Janeiro - RJ","Apartamento com 1 Quarto para Alugar, 40 m² por R$ 1.300/Mês",A melhor Black Friday de Imóveis - Sua mudança pode até sair de graça - 100% de cashback do valor de um aluguel.\n\nApartamento 1 quarto na Tijuca...,https://www.vivareal.com.br/imovel/apartamento-1-quartos-tijuca-zona-norte-rio-de-janeiro-40m2-aluguel-RS1300-id-2502350716/,"[https://resizedimgs.vivareal.com/fit-in/870x653/vr.images.sp/5aeb3dcc21d0e13b09cf7d1c835475c9.jpg, https://resizedimgs.vivareal.com/fit-in/870x65...","https://maps.google.com/maps?ll=-22.929018,-43.24337&z=16&t=m&hl=en-US&gl=US&mapclient=embed&q=R.%20Uruguai%20-%20Tijuca%20Rio%20de%20Janeiro%20-%..."
256,2503917473,vivareal,Aluguel,63,2.0,2.0,NaN,1500,600.0,-22.9175,-43.2237,"Rua Mariz e Barros, 1058 - Tijuca, Rio de Janeiro - RJ","Apartamento com 2 Quartos para Alugar, 63 m² por R$ 1.500/Mês","O apartamento está localizado no bairro da Tijuca com 63 metros quadrados com 2 quartos e 2 banheiros. Encontra-se em um condomínio fechado, porta...",https://www.vivareal.com.br/imovel/apartamento-2-quartos-tijuca-zona-norte-rio-de-janeiro-63m2-aluguel-RS1500-id-2503917473/,"[https://resizedimgs.vivareal.com/fit-in/870x653/vr.images.sp/6243b80329deefbc382f44f9810deeeb.jpg, https://resizedimgs.vivareal.com/fit-in/870x65...","https://maps.google.com/maps?ll=-22.917516,-43.223739&z=16&t=m&hl=en-US&gl=US&mapclient=embed&q=22%C2%B055%2703.1%22S%2043%C2%B013%2725.5%22W%20-2..."


In [14]:
def get_min_dist(lat, lng):
    if not pd.isna(lat):
        X = df_metro.copy()
        X["distance"] =  X[["lat", "lng"]].apply(lambda x: distance((x[0], x[1]), (lat, lng)).m, axis=1)
        return X.sort_values("distance").reset_index().loc[0,['linha', 'estacao', 'distance']].tolist()
        #return X.sort_values("distance").reset_index()
    else:
        return "", "", None

In [15]:
df[['lat', 'lng','condominio']] = df[['lat', 'lng','condominio']].apply(pd.to_numeric)

In [19]:
get_min_dist(-22.925214,-43.230616)

['Linha 1 do Metrô do Rio de Janeiro',
 'Estação Saens Peña',
 200.22080413250922]

In [6]:
df[['linha', 'estacao', 'distance']] = df[["lat", "lng"]].apply(lambda x: get_min_dist(x[0], x[1]), axis=1).apply(pd.Series)

In [18]:
df[df.url == "https://www.vivareal.com.br/imovel/apartamento-2-quartos-tijuca-zona-norte-rio-de-janeiro-65m2-aluguel-RS1550-id-92895851/?__vt=lgpd:b"]

,id,page,tipo,metragem,quartos,banheiros,vagas,preco,condominio,lat,lng,endereco,title,desc,url,imagens,url_google
248,92895851,6,Aluguel,65,2,2,0,1550,580.0,-22.925214,-43.230616,"Rua Soares da Costa, 165 - Tijuca, Rio de Janeiro - RJ","Apartamento com 2 Quartos para Alugar, 65 m² por R$ 1.550/Mês","Rua Soares da Costa, 165: Apartamento em Excelente Estado, Composto por Sala, 02 Quartos, Banheiro Social, Copa-Cozinha c/Armários, Área e Banheir...",https://www.vivareal.com.br/imovel/apartamento-2-quartos-tijuca-zona-norte-rio-de-janeiro-65m2-aluguel-RS1550-id-92895851/?__vt=lgpd:b,"[https://resizedimgs.vivareal.com/fit-in/870x653/vr.images.sp/172c80c58e308373798e3b9f0eb034ac.jpg, https://resizedimgs.vivareal.com/crop/142x80/v...","https://maps.google.com/maps?ll=-22.925214,-43.230616&z=16&t=m&hl=en-US&gl=US&mapclient=embed&q=22%C2%B055%2730.8%22S%2043%C2%B013%2750.2%22W%20-2..."


In [23]:
df[~df.lat.isna()]

,id,page,tipo,metragem,quartos,banheiros,vagas,preco,condominio,lat,lng,endereco,title,desc,url,imagens,url_google
0,2482894253,1,Aluguel,84,2,2,1,2100,680.0,-22.922354,-43.228156,"Rua Conselheiro Zenha, 49 - Tijuca, Rio de Janeiro - RJ","Apartamento com 2 Quartos para Alugar, 84 m² por R$ 2.100/Mês","EXCELENTE APARTAMENTO, AMPLOS QUARTOS COM ARMARIOS EMBUTIDOS, BANHEIRO COM ARMARIOS E BLINDEX, AREA DE SERVIÇO SEPARADA COM QUARTO E BANHEIRO DE E...",https://www.vivareal.com.br/imovel/apartamento-2-quartos-tijuca-zona-norte-rio-de-janeiro-com-garagem-84m2-aluguel-RS2100-id-2482894253/?__vt=lgpd:b,"[https://resizedimgs.vivareal.com/fit-in/870x653/vr.images.sp/798ce6de0e00a9333fe6e0c790e60446.jpg, https://resizedimgs.vivareal.com/crop/142x80/v...","https://maps.google.com/maps?ll=-22.922354,-43.228156&z=16&t=m&hl=en-US&gl=US&mapclient=embed&q=22%C2%B055%2720.5%22S%2043%C2%B013%2741.4%22W%20-2..."
1,2168785752,1,Aluguel,100,2,3,1,1800,974.0,-22.932573,-43.241025,"Rua Barão de Mesquita, 574 - Tijuca, Rio de Janeiro - RJ","Apartamento com 2 Quartos para Alugar, 100 m² por R$ 1.800/Mês","RUA BARÃO DE MESQUITA 574 - Frente, sol da manhã, com 100m2 e vaga de garagem. Sala, 2 quartos, sendo 1 suíte com closet, banheiro social, varanda...",https://www.vivareal.com.br/imovel/apartamento-2-quartos-tijuca-zona-norte-rio-de-janeiro-com-garagem-100m2-aluguel-RS1800-id-2168785752/?__vt=lgpd:b,"[https://resizedimgs.vivareal.com/fit-in/870x653/vr.images.sp/ba1001d761161566ab30fe9d26a4dedd.jpg, https://resizedimgs.vivareal.com/crop/142x80/v...","https://maps.google.com/maps?ll=-22.932573,-43.241025&z=16&t=m&hl=en-US&gl=US&mapclient=embed&q=22%C2%B055%2757.3%22S%2043%C2%B014%2727.7%22W%20-2..."
2,2487879139,1,Aluguel,51,1,1,1,1600,710.0,-22.928453,-43.240066,"Avenida Maracanã - Tijuca, Rio de Janeiro - RJ","Apartamento com 1 Quarto para Alugar, 51 m² por R$ 1.600/Mês","Rua Padre Champagnat - ótimo apto. 51m2, varanda, suíte, deps. completas, piso frio, área de serviço, 01 vaga, sol manhã, ótima localização, junto...",https://www.vivareal.com.br/imovel/apartamento-1-quartos-tijuca-zona-norte-rio-de-janeiro-com-garagem-51m2-aluguel-RS1600-id-2487879139/?__vt=lgpd:b,"[https://resizedimgs.vivareal.com/fit-in/870x653/vr.images.sp/898c45ba109a009ceff92f8afc5b4101.jpg, https://resizedimgs.vivareal.com/crop/142x80/v...","https://maps.google.com/maps?ll=-22.928453,-43.240066&z=16&t=m&hl=en-US&gl=US&mapclient=embed&q=Av.%20Maracan%C3%A3%20-%20Tijuca%20Rio%20de%20Jane..."
4,2459914673,4,Aluguel,30,2,1,0,2000,0.0,-22.924242,-43.242506,"Rua Barão de Mesquita - Tijuca, Rio de Janeiro - RJ","Apartamento com 2 Quartos para Alugar, 30 m² por R$ 2.000/Mês",,https://www.vivareal.com.br/imovel/apartamento-2-quartos-tijuca-zona-norte-rio-de-janeiro-30m2-aluguel-RS2000-id-2459914673/?__vt=lgpd:b,"[https://resizedimgs.vivareal.com/fit-in/870x653/vr.images.sp/f40bab5860ea1489c673b745afeccf56.jpg, https://resizedimgs.vivareal.com/crop/142x80/v...","https://maps.google.com/maps?ll=-22.924242,-43.242506&z=16&t=m&hl=en-US&gl=US&mapclient=embed&q=R.%20Bar%C3%A3o%20de%20Mesquita%20-%20Tijuca%20Rio..."
5,2497880948,4,Aluguel,70,1,1,0,1500,650.0,-22.926095,-43.231480,"Rua General Roca - Tijuca, Rio de Janeiro - RJ","Apartamento com 1 Quarto para Alugar, 70 m² por R$ 1.500/Mês",APARTAMENTO JUNTO À PRAÇA SÃENS PEÑA!,https://www.vivareal.com.br/imovel/apartamento-1-quartos-tijuca-zona-norte-rio-de-janeiro-70m2-aluguel-RS1500-id-2497880948/?__vt=lgpd:b,[],"https://maps.google.com/maps?ll=-22.926095,-43.23148&z=16&t=m&hl=en-US&gl=US&mapclient=embed&q=R.%20Gen.%20Roca%20-%20Tijuca%20Rio%20de%20Janeiro%..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,2495806924,1,Aluguel,62,2,2,0,1800,427.0,-22.925975,-43.231254,"Rua General Roca, 490 - Tijuca, Rio de Janeiro - RJ","Apartamento com 2 Quartos para Alugar, 62 m² por R$ 1.800/Mês","Vista para o Maciço da Tijuca, 7o. andar, sem qualquer vista para comunidade, sala, dois quartos, banheiro social elegante com 

In [8]:
df.groupby("page")["id"].count()

page
1     36
2     36
3     36
4     34
5     34
6     36
9     35
10    33
Name: id, dtype: int64

In [9]:
df.groupby("tipo")["id"].count()

tipo
Aluguel    250
Compra      30
Name: id, dtype: int64

In [10]:
df = df[df.tipo == "Aluguel"]
df.head(3)

,id,page,tipo,metragem,quartos,banheiros,vagas,preco,condominio,lat,lng,endereco,title,desc,url,imagens,url_google,linha,estacao,distance
0,2482894253,1,Aluguel,84,2,2,1,2100,680.0,-22.922354,-43.228156,"Rua Conselheiro Zenha, 49 - Tijuca, Rio de Janeiro - RJ","Apartamento com 2 Quartos para Alugar, 84 m² por R$ 2.100/Mês","EXCELENTE APARTAMENTO, AMPLOS QUARTOS COM ARMARIOS EMBUTIDOS, BANHEIRO COM ARMARIOS E BLINDEX, AREA DE SERVIÇO SEPARADA COM QUARTO E BANHEIRO DE E...",https://www.vivareal.com.br/imovel/apartamento-2-quartos-tijuca-zona-norte-rio-de-janeiro-com-garagem-84m2-aluguel-RS2100-id-2482894253/?__vt=lgpd:b,"[https://resizedimgs.vivareal.com/fit-in/870x653/vr.images.sp/798ce6de0e00a9333fe6e0c790e60446.jpg, https://resizedimgs.vivareal.com/crop/142x80/v...","https://maps.google.com/maps?ll=-22.922354,-43.228156&z=16&t=m&hl=en-US&gl=US&mapclient=embed&q=22%C2%B055%2720.5%22S%2043%C2%B013%2741.4%22W%20-2...",Linha 1 do Metrô do Rio de Janeiro,Estação São Francisco Xavier (Metrô Rio),480.877012
1,2168785752,1,Aluguel,100,2,3,1,1800,974.0,-22.932573,-43.241025,"Rua Barão de Mesquita, 574 - Tijuca, Rio de Janeiro - RJ","Apartamento com 2 Quartos para Alugar, 100 m² por R$ 1.800/Mês","RUA BARÃO DE MESQUITA 574 - Frente, sol da manhã, com 100m2 e vaga de garagem. Sala, 2 quartos, sendo 1 suíte com closet, banheiro social, varanda...",https://www.vivareal.com.br/imovel/apartamento-2-quartos-tijuca-zona-norte-rio-de-janeiro-com-garagem-100m2-aluguel-RS1800-id-2168785752/?__vt=lgpd:b,"[https://resizedimgs.vivareal.com/fit-in/870x653/vr.images.sp/ba1001d761161566ab30fe9d26a4dedd.jpg, https://resizedimgs.vivareal.com/crop/142x80/v...","https://maps.google.com/maps?ll=-22.932573,-43.241025&z=16&t=m&hl=en-US&gl=US&mapclient=embed&q=22%C2%B055%2757.3%22S%2043%C2%B014%2727.7%22W%20-2...",Linha 1 do Metrô do Rio de Janeiro,Estação Saens Peña,1338.858363
2,2487879139,1,Aluguel,51,1,1,1,1600,710.0,-22.928453,-43.240066,"Avenida Maracanã - Tijuca, Rio de Janeiro - RJ","Apartamento com 1 Quarto para Alugar, 51 m² por R$ 1.600/Mês","Rua Padre Champagnat - ótimo apto. 51m2, varanda, suíte, deps. completas, piso frio, área de serviço, 01 vaga, sol manhã, ótima localização, junto...",https://www.vivareal.com.br/imovel/apartamento-1-quartos-tijuca-zona-norte-rio-de-janeiro-com-garagem-51m2-aluguel-RS1600-id-2487879139/?__vt=lgpd:b,"[https://resizedimgs.vivareal.com/fit-in/870x653/vr.images.sp/898c45ba109a009ceff92f8afc5b4101.jpg, https://resizedimgs.vivareal.com/crop/142x80/v...","https://maps.google.com/maps?ll=-22.928453,-43.240066&z=16&t=m&hl=en-US&gl=US&mapclient=embed&q=Av.%20Maracan%C3%A3%20-%20Tijuca%20Rio%20de%20Jane...",Linha 1 do Metrô do Rio de Janeiro,Estação Saens Peña,974.465523


In [12]:
df["preco_total"] = df["preco"] + df["condominio"]

In [14]:
df.head(3)

,id,page,tipo,metragem,quartos,banheiros,vagas,preco,condominio,lat,...,endereco,title,desc,url,imagens,url_google,linha,estacao,distance,preco_total
0,2482894253,1,Aluguel,84,2,2,1,2100,680.0,-22.922354,...,"Rua Conselheiro Zenha, 49 - Tijuca, Rio de Janeiro - RJ","Apartamento com 2 Quartos para Alugar, 84 m² por R$ 2.100/Mês","EXCELENTE APARTAMENTO, AMPLOS QUARTOS COM ARMARIOS EMBUTIDOS, BANHEIRO COM ARMARIOS E BLINDEX, AREA DE SERVIÇO SEPARADA COM QUARTO E BANHEIRO DE E...",https://www.vivareal.com.br/imovel/apartamento-2-quartos-tijuca-zona-norte-rio-de-janeiro-com-garagem-84m2-aluguel-RS2100-id-2482894253/?__vt=lgpd:b,"[https://resizedimgs.vivareal.com/fit-in/870x653/vr.images.sp/798ce6de0e00a9333fe6e0c790e60446.jpg, https://resizedimgs.vivareal.com/crop/142x80/v...","https://maps.google.com/maps?ll=-22.922354,-43.228156&z=16&t=m&hl=en-US&gl=US&mapclient=embed&q=22%C2%B055%2720.5%22S%2043%C2%B013%2741.4%22W%20-2...",Linha 1 do Metrô do Rio de Janeiro,Estação São Francisco Xavier (Metrô Rio),480.877012,2780.0
1,2168785752,1,Aluguel,100,2,3,1,1800,974.0,-22.932573,...,"Rua Barão de Mesquita, 574 - Tijuca, Rio de Janeiro - RJ","Apartamento com 2 Quartos para Alugar, 100 m² por R$ 1.800/Mês","RUA BARÃO DE MESQUITA 574 - Frente, sol da manhã, com 100m2 e vaga de garagem. Sala, 2 quartos, sendo 1 suíte com closet, banheiro social, varanda...",https://www.vivareal.com.br/imovel/apartamento-2-quartos-tijuca-zona-norte-rio-de-janeiro-com-garagem-100m2-aluguel-RS1800-id-2168785752/?__vt=lgpd:b,"[https://resizedimgs.vivareal.com/fit-in/870x653/vr.images.sp/ba1001d761161566ab30fe9d26a4dedd.jpg, https://resizedimgs.vivareal.com/crop/142x80/v...","https://maps.google.com/maps?ll=-22.932573,-43.241025&z=16&t=m&hl=en-US&gl=US&mapclient=embed&q=22%C2%B055%2757.3%22S%2043%C2%B014%2727.7%22W%20-2...",Linha 1 do Metrô do Rio de Janeiro,Estação Saens Peña,1338.858363,2774.0
2,2487879139,1,Aluguel,51,1,1,1,1600,710.0,-22.928453,...,"Avenida Maracanã - Tijuca, Rio de Janeiro - RJ","Apartamento com 1 Quarto para Alugar, 51 m² por R$ 1.600/Mês","Rua Padre Champagnat - ótimo apto. 51m2, varanda, suíte, deps. completas, piso frio, área de serviço, 01 vaga, sol manhã, ótima localização, junto...",https://www.vivareal.com.br/imovel/apartamento-1-quartos-tijuca-zona-norte-rio-de-janeiro-com-garagem-51m2-aluguel-RS1600-id-2487879139/?__vt=lgpd:b,"[https://resizedimgs.vivareal.com/fit-in/870x653/vr.images.sp/898c45ba109a009ceff92f8afc5b4101.jpg, https://resizedimgs.vivareal.com/crop/142x80/v...","https://maps.google.com/maps?ll=-22.928453,-43.240066&z=16&t=m&hl=en-US&gl=US&mapclient=embed&q=Av.%20Maracan%C3%A3%20-%20Tijuca%20Rio%20de%20Jane...",Linha 1 do Metrô do Rio de Janeiro,Estação Saens Peña,974.465523,2310.0
